In [2]:
from binance.client import Client
import pandas as pd
import time
from datetime import datetime
from dotenv import load_dotenv
import os
# Initialize Binance client with API keys
# You need to create API keys in your Binance account settings
def initialize_binance_client(api_key, api_secret):
    """
    Initialize and return a Binance client using API credentials
    """
    client = Client(api_key, api_secret)
    return client
# Example API credentials (replace with your own)
# WARNING: Never share your API keys or commit them to version control
load_dotenv()
api_key = os.getenv('BinanceRestrictedAPIKey')
api_secret = os.getenv('BinanceRestrictedAPISecret')



In [3]:
def execute_arbitrage(client, opportunity, investment_amount=100):
    """
    Execute an arbitrage opportunity
    
    Parameters:
    - client: Binance client instance
    - opportunity: Row from our arbitrage dataframe
    - investment_amount: Amount to invest in USDT
    
    Returns:
    - Execution details
    """
    coin = opportunity['coin']
    best_path = opportunity['best_path']
    profit_percentage = opportunity['profit_percentage']
    
    print(f"Executing arbitrage for {coin} via {best_path}")
    print(f"Expected profit: {profit_percentage:.4f}%")
    
    # Implementation would depend on the specific path
    # This is a simplified example
    if best_path == 'Via BTC':
        # 1. Buy BTC with USDT
        # 2. Buy target coin with BTC
        # 3. Sell target coin for USDT
        pass
    elif best_path == 'Via ETH':
        # Similar steps using ETH as intermediate
        pass
    
    # In a real implementation, you would:
    # 1. Calculate exact quantities based on current prices
    # 2. Execute the trades in sequence
    # 3. Handle errors and edge cases
    # 4. Verify the final profit
    
    return {
        "coin": coin,
        "path": best_path,
        "expected_profit": profit_percentage,
        "timestamp": datetime.now().strftime("%H:%M:%S")
    }

# Function to place a market order
def place_market_order(client, symbol, side, quantity):
    """
    Place a market order on Binance
    
    Parameters:
    - client: Binance client instance
    - symbol: Trading pair (e.g., 'BTCUSDT')
    - side: 'BUY' or 'SELL'
    - quantity: Amount to buy/sell
    
    Returns:
    - Order information
    """
    try:
        order = client.create_order(
            symbol=symbol,
            side=side,
            type='MARKET',
            quantity=quantity
        )
        return order
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to place a limit order
def place_limit_order(client, symbol, side, quantity, price):
    """
    Place a limit order on Binance
    
    Parameters:
    - client: Binance client instance
    - symbol: Trading pair (e.g., 'BTCUSDT')
    - side: 'BUY' or 'SELL'
    - quantity: Amount to buy/sell
    - price: Limit price
    
    Returns:
    - Order information
    """
    try:
        order = client.create_order(
            symbol=symbol,
            side=side,
            type='LIMIT',
            timeInForce='GTC',  # Good Till Cancelled
            quantity=quantity,
            price=price
        )
        return order
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to get account balance
def get_account_balance(client, asset=None):
    """
    Get account balance for a specific asset or all assets
    
    Parameters:
    - client: Binance client instance
    - asset: Asset symbol (e.g., 'BTC') or None for all assets
    
    Returns:
    - Balance information
    """
    try:
        account_info = client.get_account()
        balances = account_info['balances']
        
        if asset:
            for balance in balances:
                if balance['asset'] == asset:
                    return balance
            return None
        else:
            # Return non-zero balances
            return [b for b in balances if float(b['free']) > 0 or float(b['locked']) > 0]
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage (commented out for safety)
"""
# Initialize client
client = initialize_binance_client(api_key, api_secret)

# Check account balance
btc_balance = get_account_balance(client, 'BTC')
print(f"BTC Balance: {btc_balance}")

# Place a market buy order
# order = place_market_order(client, 'BTCUSDT', 'BUY', 0.001)
# print(f"Order placed: {order}")

# Place a limit sell order
# order = place_limit_order(client, 'BTCUSDT', 'SELL', 0.001, 85000)
# print(f"Limit order placed: {order}")
"""

client = initialize_binance_client(api_key, api_secret)
cryptos = get_account_balance(client)

import json
info = client.get_symbol_info('ONEUSDT')
print(json.dumps(info, indent=4))
order = place_limit_order(client, 'ONEUSDT', 'SELL', 50, 0.1)
print(f"Limit order placed: {order}")

# Function to execute arbitrage based on our data


{
    "symbol": "ONEUSDT",
    "status": "TRADING",
    "baseAsset": "ONE",
    "baseAssetPrecision": 8,
    "quoteAsset": "USDT",
    "quotePrecision": 8,
    "quoteAssetPrecision": 8,
    "baseCommissionPrecision": 8,
    "quoteCommissionPrecision": 8,
    "orderTypes": [
        "LIMIT",
        "LIMIT_MAKER",
        "MARKET",
        "STOP_LOSS",
        "STOP_LOSS_LIMIT",
        "TAKE_PROFIT",
        "TAKE_PROFIT_LIMIT"
    ],
    "icebergAllowed": true,
    "ocoAllowed": true,
    "otoAllowed": true,
    "quoteOrderQtyMarketAllowed": true,
    "allowTrailingStop": true,
    "cancelReplaceAllowed": true,
    "isSpotTradingAllowed": true,
    "isMarginTradingAllowed": true,
    "filters": [
        {
            "filterType": "PRICE_FILTER",
            "minPrice": "0.00001000",
            "maxPrice": "1000.00000000",
            "tickSize": "0.00001000"
        },
        {
            "filterType": "LOT_SIZE",
            "minQty": "0.10000000",
            "maxQty": "9000000

In [4]:
import math
current_price = float(client.get_symbol_ticker(symbol="ONEUSDT")['price'])
print(f"Current ONEUSDT price: {current_price}")
print(f"Max ONEUSDT price: {current_price*5}")
# For a sell order slightly above market
sell_price = round(current_price * 1.01, 5)  # 1% above current price

# Calculate required quantity to meet minimum notional of 5 USDT
min_quantity = math.ceil(5 / sell_price / 0.1) * 0.1  # Round up to nearest 0.1
print(f"Min quantity: {min_quantity}")
print(f"Sell price: {sell_price}")
# order = place_limit_order(client, 'ONEUSDT', 'SELL', round(min_quantity,2), sell_price)

Current ONEUSDT price: 0.0114
Max ONEUSDT price: 0.057
Min quantity: 434.5
Sell price: 0.01151


In [55]:
def execute_triangular_arbitrage(client, start_amount, path):
    """
    Execute a triangular arbitrage trade following a specific path
    
    Args:
        client: Binance client instance
        start_amount: Amount of starting currency (e.g., USDT)
        path: List of trading pairs to follow (e.g., ["ETHUSDT", "ETHBTC", "BTCUSDT"])
    
    Returns:
        final_amount: The final amount after completing the arbitrage
    """
    print(f"Starting triangular arbitrage with {start_amount} {path[0][-4:]}")
    print(f"Path: {' -> '.join(path)}")
    
    # Step 1: Buy first asset with starting currency
    first_pair = path[0]
    first_price = float(client.get_symbol_ticker(symbol=first_pair)['price'])
    first_asset = first_pair[:-4]  # Extract base asset from pair
    first_quantity = start_amount / first_price
    
    # Get symbol info to ensure we meet minimum notional and lot size requirements
    symbol_info = client.get_symbol_info(first_pair)
    lot_size_filter = next((f for f in symbol_info['filters'] if f['filterType'] == 'LOT_SIZE'), None)
    
    if lot_size_filter:
        step_size = float(lot_size_filter['stepSize'])
        first_quantity = math.floor(first_quantity / step_size) * step_size
    
    print(f"Buying {first_quantity} {first_asset} at {first_price} {path[0][-4:]}")
    buy_order = place_market_order(client, first_pair, 'BUY', first_quantity)
    print(f"Buy order executed: {buy_order}")
    
    # Get actual filled quantity
    filled_quantity = float(buy_order['executedQty'])
    print(f"Received {filled_quantity} {first_asset}")
    
    # Step 2: Sell first asset for second asset
    second_pair = path[1]
    second_price = float(client.get_symbol_ticker(symbol=second_pair)['price'])
    second_asset = second_pair[-3:] if second_pair.endswith('BTC') else second_pair[-4:]
    
    # Get symbol info for second pair
    symbol_info = client.get_symbol_info(second_pair)
    lot_size_filter = next((f for f in symbol_info['filters'] if f['filterType'] == 'LOT_SIZE'), None)
    
    if lot_size_filter:
        step_size = float(lot_size_filter['stepSize'])
        filled_quantity = math.floor(filled_quantity / step_size) * step_size
    
    print(f"Selling {filled_quantity} {first_asset} for {second_asset} at {second_price} {second_asset} per {first_asset}")
    sell_order = place_market_order(client, second_pair, 'SELL', filled_quantity)
    print(f"Sell order executed: {sell_order}")
    
    # Get second asset received
    second_received = float(sell_order['cummulativeQuoteQty'])
    print(f"Received {second_received} {second_asset}")
    
    # Step 3: Sell second asset for final currency
    third_pair = path[2]
    third_price = float(client.get_symbol_ticker(symbol=third_pair)['price'])
    final_currency = third_pair[-4:]
    
    print(f"Selling {second_received} {second_asset} for {final_currency} at {third_price} {final_currency} per {second_asset}")
    final_order = place_market_order(client, third_pair, 'SELL', second_received)
    print(f"Final order executed: {final_order}")
    
    # Calculate final amount
    final_amount = float(final_order['cummulativeQuoteQty'])
    profit_percentage = ((final_amount - start_amount) / start_amount) * 100
    
    print(f"Arbitrage complete!")
    print(f"Started with: {start_amount} {path[0][-4:]}")
    print(f"Ended with: {final_amount} {final_currency}")
    print(f"Profit: {final_amount - start_amount} {final_currency} ({profit_percentage:.2f}%)")
    
    return final_amount

# Example usage for GALA triangular arbitrage
# Uncomment to execute (be careful, this will place real orders!)

start_amount = 10  # USDT
path = ["STEEMUSDT", "STEEMBTC", "BTCUSDT"]
final_amount = execute_triangular_arbitrage(client, start_amount, path)

# Simulate 100 iterations (this is just a calculation, not actual trades)
# iterations = 10
# compound_amount = start_amount
# for i in range(iterations):
#     compound_amount = compound_amount * (1 + 0.0299)  # 2.99% profit per iteration

# print(f"After {iterations} iterations:")
# print(f"Final Balance: {compound_amount:.2f} USDT")
# print(f"Total Profit: {compound_amount - start_amount:.2f} USDT ({(compound_amount/start_amount - 1)*100:.2f}%)")


Starting triangular arbitrage with 10 USDT
Path: STEEMUSDT -> STEEMBTC -> BTCUSDT
Buying 75.10000000000001 STEEM at 0.133 USDT
An error occurred: APIError(code=-2010): Symbol not whitelisted for API key.
Buy order executed: None


TypeError: 'NoneType' object is not subscriptable

In [131]:

def format_quantity(client, symbol, quantity):
    """
    Format a quantity to comply with the LOT_SIZE filter for a given symbol.
    
    Args:
        client: Binance client instance
        symbol: Trading pair symbol (e.g., 'BTCUSDT')
        quantity: The original quantity to format
        
    Returns:
        float: The quantity formatted according to the symbol's LOT_SIZE requirements
    """
    # Get symbol information
    info = client.get_symbol_info(symbol)
    
    # Find the LOT_SIZE filter
    lot_size_filter = next((f for f in info['filters'] if f['filterType'] == 'LOT_SIZE'), None)
    
    if lot_size_filter:
        # Extract step size
        step_size = float(lot_size_filter['stepSize'])
        
        # Calculate precision based on step size
        precision = 0
        if step_size < 1:
            precision = len(str(step_size).split('.')[-1].rstrip('0'))
        
        # Format the quantity to be a multiple of the step size
        formatted_quantity = math.floor(float(quantity) / step_size) * step_size
        
        # Round to the appropriate precision
        formatted_quantity = round(formatted_quantity, precision)
        
        return formatted_quantity
    else:
        # If no LOT_SIZE filter found, return the original quantity
        return float(quantity)



In [189]:
# Define the initial investment and trading path
initial_investment = 100  # USDT
asset = 'STEEM'
route = 'BTC'
client = Client(api_key, api_secret)

# Define the complete trading path
path = [f"{asset}USDT", f"{asset}{route}", f"{route}USDT"]

# Step 1: Buy asset with USDT
asset_price = float(client.get_symbol_ticker(symbol=path[0])['price'])
asset_quantity = initial_investment / asset_price
formatted_asset_quantity = float(format_quantity(client, symbol=path[0], quantity=asset_quantity))
print(f"Estimated {asset} quantity: {asset_quantity}")
print(f"Formatted {asset} quantity to buy: {formatted_asset_quantity}")

# Simulate the first trade (USDT -> STEEM)
# In a real scenario, this would be: buy_order = client.create_order(symbol=path[0], side='BUY', type='MARKET', quantity=formatted_asset_quantity)
# For simulation, assume we get exactly what we formatted
act_asset_bought = formatted_asset_quantity
print(f"Bought {act_asset_bought} {asset}")

# Step 2: Sell asset for route currency (STEEM -> BTC)
asset_to_route_price = float(client.get_symbol_ticker(symbol=path[1])['price'])
route_quantity = act_asset_bought * asset_to_route_price
formatted_route_quantity = format_quantity(client, symbol=path[1], quantity=route_quantity)
print(f"Estimated {route} quantity: {route_quantity}")
print(f"Formatted {route} quantity after selling {asset}: {formatted_route_quantity}")

# Step 3: Sell route currency for USDT (BTC -> USDT)
route_to_usdt_price = float(client.get_symbol_ticker(symbol=path[2])['price'])
final_usdt = formatted_route_quantity * route_to_usdt_price
print(f"Final USDT amount: {final_usdt}")
print(f"Profit/Loss: {final_usdt - initial_investment} USDT ({((final_usdt/initial_investment)-1)*100:.2f}%)")

Estimated STEEM quantity: 756.42965204236
Formatted STEEM quantity to buy: 756.4
Bought 756.4 STEEM
Estimated BTC quantity: 0.001202676
Formatted BTC quantity after selling STEEM: 0
Final USDT amount: 0.0
Profit/Loss: -100.0 USDT (-100.00%)


In [8]:
def format_quantity(client, symbol, quantity):
    """
    Format a quantity to comply with the LOT_SIZE filter for a given symbol.
    
    Args:
        client: Binance client instance
        symbol: Trading pair symbol (e.g., 'BTCUSDT')
        quantity: The original quantity to format
        
    Returns:
        float: The quantity formatted according to the symbol's LOT_SIZE requirements
    """
    # Get symbol information
    info = client.get_symbol_info(symbol)
    
    # Find the LOT_SIZE filter
    lot_size_filter = next((f for f in info['filters'] if f['filterType'] == 'LOT_SIZE'), None)
    
    if lot_size_filter:
        # Extract step size
        step_size = float(lot_size_filter['stepSize'])
        
        # Calculate precision based on step size
        precision = 0
        if step_size < 1:
            precision = len(str(step_size).split('.')[-1].rstrip('0'))
        
        # Format the quantity to be a multiple of the step size
        formatted_quantity = math.floor(float(quantity) / step_size) * step_size
        
        # Round to the appropriate precision
        formatted_quantity = round(formatted_quantity, precision)
        
        return formatted_quantity
    else:
        # If no LOT_SIZE filter found, return the original quantity
        return float(quantity)

def execute_triangular_trade(client, investment, asset, route, iterations=1):
    """
    Execute triangular arbitrage trades on Binance
    
    Args:
        client: Binance client instance
        investment: Amount to invest in USDT (if None, uses available balance minus 1 USDT)
        asset: The crypto asset to trade (e.g., 'GALA')
        route: The intermediate currency (e.g., 'ETH')
        iterations: Number of consecutive iterations to execute
        
    Returns:
        list: Results of each iteration including starting and final balances
    """
    results = []
    
    for i in range(iterations):
        iteration_result = {}
        
        # Get current USDT balance
        usdt_balance = float(client.get_asset_balance('USDT')['free'])
        print(f"Current balance in USDT: {usdt_balance}")
        
        # Determine investment amount
        if investment is None:
            investment_amount = math.floor(usdt_balance - 1)
        else:
            investment_amount = investment
            
        print(f"Investment amount: {investment_amount}")
        iteration_result['starting_balance'] = usdt_balance
        iteration_result['investment'] = investment_amount
        
        # Define trading path
        path = [f"{asset}USDT", f"{asset}{route}", f"{route}USDT"]
        
        # Step 1: Buy asset with USDT
        asset_price = float(client.get_symbol_ticker(symbol=path[0])['price'])
        asset_quantity = round(investment_amount / asset_price, 1)
        formatted_quantity = format_quantity(client, path[0], asset_quantity)
        print(f"{asset} quantity buying: {formatted_quantity}")
        buy1 = client.create_order(symbol=path[0], side='BUY', type='MARKET', quantity=formatted_quantity)
        iteration_result['buy_order'] = buy1

        # Get actual asset quantity received
        asset_quantity = client.get_asset_balance(asset=asset)['free']
        print(f"{asset} quantity bought: {asset_quantity}")

        # Step 2: Sell asset for route currency
        formatted_asset_quantity = format_quantity(client, path[1], float(asset_quantity))
        print(f"Formatted {asset} quantity to sell: {formatted_asset_quantity}")
        sell1 = client.create_order(symbol=path[1], side='SELL', type='MARKET', quantity=formatted_asset_quantity)
        iteration_result['sell_order_1'] = sell1

        # Step 3: Sell route currency for USDT
        route_quantity = client.get_asset_balance(asset=route)['free']
        print(f"Original {route} quantity: {route_quantity}")
        formatted_route_quantity = format_quantity(client, path[2], float(route_quantity))
        print(f"Formatted {route} quantity: {formatted_route_quantity}")
        sell2 = client.create_order(
            symbol=path[2], 
            side='SELL', 
            type='MARKET', 
            quantity=formatted_route_quantity
        )
        iteration_result['sell_order_2'] = sell2
        
        # Get final USDT balance
        final_usdt = float(client.get_asset_balance('USDT')['free'])
        print(f"Returned balance in USDT: {final_usdt}")
        
        # Calculate profit/loss
        profit = final_usdt - iteration_result['starting_balance']
        profit_percentage = (profit / investment_amount) * 100
        print(f"Profit/Loss: {profit} USDT ({profit_percentage:.2f}%)")
        
        iteration_result['final_balance'] = final_usdt
        iteration_result['profit'] = profit
        iteration_result['profit_percentage'] = profit_percentage
        
        results.append(iteration_result)
    
    return results

# Example usage:
client = Client(api_key, api_secret)
results = execute_triangular_trade(client, 
                                   investment=None, 
                                   asset='GALA', 
                                   route='BTC', 
                                   iterations=30)


Current balance in USDT: 9.95512667
Investment amount: 8
GALA quantity buying: 474.0
GALA quantity bought: 474.00000000
Formatted GALA quantity to sell: 474.0


BinanceAPIException: APIError(code=-1013): Filter failure: NOTIONAL

In [201]:
def simulate_triangular_arbitrage(client, investment, asset, route, include_fees=True, iterations=1):
    """
    Simulate a triangular arbitrage trade to analyze the impact of exchange filters
    
    Args:
        client: Binance client instance
        investment: Initial investment amount in USDT
        asset: The crypto asset to trade (e.g., 'STEEM')
        route: The intermediate currency (e.g., 'BTC')
        include_fees: Whether to include trading fees in the simulation (default: True)
        iterations: Number of consecutive iterations to simulate (default: 1)
        
    Returns:
        dict: Detailed results of the simulation including impact of formatting
    """
    # Define trading fee (0.1% per trade for standard users)
    fee_rate = 0.001 if include_fees else 0
    
    # Define the complete trading path
    path = [f"{asset}USDT", f"{asset}{route}", f"{route}USDT"]
    
    results = {
        "investment": investment,
        "asset": asset,
        "route": route,
        "path": path,
        "steps": [],
        "iterations": [],
        "summary": {}
    }
    
    current_investment = investment
    
    for iteration in range(1, iterations + 1):
        iteration_result = {
            "iteration": iteration,
            "starting_amount": current_investment,
            "steps": []
        }
        
        # Step 1: Buy asset with USDT
        asset_price = float(client.get_symbol_ticker(symbol=path[0])['price'])
        ideal_asset_quantity = current_investment / asset_price
        formatted_asset_quantity = format_quantity(client, path[0], ideal_asset_quantity)
        
        # Calculate actual cost after formatting
        actual_cost = formatted_asset_quantity * asset_price
        # Calculate fee impact
        fee_impact = formatted_asset_quantity * fee_rate
        # Apply trading fee
        asset_received = formatted_asset_quantity * (1 - fee_rate)
        
        step1 = {
            "operation": f"Buy {asset} with USDT",
            "price": asset_price,
            "ideal_quantity": ideal_asset_quantity,
            "formatted_quantity": formatted_asset_quantity,
            "quantity_lost_to_format": ideal_asset_quantity - formatted_asset_quantity,
            "format_impact_percentage": ((ideal_asset_quantity - formatted_asset_quantity) / ideal_asset_quantity) * 100 if ideal_asset_quantity > 0 else 0,
            "cost": actual_cost,
            "fee": fee_impact if include_fees else 0,
            "fee_impact_percentage": (fee_impact / formatted_asset_quantity) * 100 if formatted_asset_quantity > 0 else 0,
            "asset_received": asset_received
        }
        
        if iteration == 1:
            results["steps"].append(step1)
        iteration_result["steps"].append(step1)
        
        # Step 2: Sell asset for route currency
        asset_to_route_price = float(client.get_symbol_ticker(symbol=path[1])['price'])
        ideal_route_quantity = asset_received * asset_to_route_price
        
        # Format the quantity of the asset being sold
        formatted_asset_sell = format_quantity(client, path[1], asset_received)
        # Calculate fee impact
        fee_impact_route = formatted_asset_sell * asset_to_route_price * fee_rate
        # Calculate actual amount of route currency received
        route_received = formatted_asset_sell * asset_to_route_price * (1 - fee_rate)
        
        step2 = {
            "operation": f"Sell {asset} for {route}",
            "price": asset_to_route_price,
            "ideal_asset_to_sell": asset_received,
            "formatted_asset_to_sell": formatted_asset_sell,
            "asset_lost_to_format": asset_received - formatted_asset_sell,
            "format_impact_percentage": ((asset_received - formatted_asset_sell) / asset_received) * 100 if asset_received > 0 else 0,
            "ideal_route_quantity": ideal_route_quantity,
            "route_received": route_received,
            "fee": fee_impact_route if include_fees else 0,
            "fee_impact_percentage": (fee_impact_route / (formatted_asset_sell * asset_to_route_price)) * 100 if formatted_asset_sell * asset_to_route_price > 0 else 0
        }
        
        if iteration == 1:
            results["steps"].append(step2)
        iteration_result["steps"].append(step2)
        
        # Step 3: Sell route currency for USDT
        route_to_usdt_price = float(client.get_symbol_ticker(symbol=path[2])['price'])
        ideal_usdt_return = route_received * route_to_usdt_price
        
        # Format the quantity of the route currency being sold
        formatted_route_sell = format_quantity(client, path[2], route_received)
        # Calculate fee impact
        fee_impact_usdt = formatted_route_sell * route_to_usdt_price * fee_rate
        # Calculate final USDT received
        final_usdt = formatted_route_sell * route_to_usdt_price * (1 - fee_rate)
        
        step3 = {
            "operation": f"Sell {route} for USDT",
            "price": route_to_usdt_price,
            "ideal_route_to_sell": route_received,
            "formatted_route_to_sell": formatted_route_sell,
            "route_lost_to_format": route_received - formatted_route_sell,
            "format_impact_percentage": ((route_received - formatted_route_sell) / route_received) * 100 if route_received > 0 else 0,
            "ideal_usdt_return": ideal_usdt_return,
            "usdt_received": final_usdt,
            "fee": fee_impact_usdt if include_fees else 0,
            "fee_impact_percentage": (fee_impact_usdt / (formatted_route_sell * route_to_usdt_price)) * 100 if formatted_route_sell * route_to_usdt_price > 0 else 0
        }
        
        if iteration == 1:
            results["steps"].append(step3)
        iteration_result["steps"].append(step3)
        
        # Calculate iteration results
        profit = final_usdt - current_investment
        profit_percentage = (profit / current_investment) * 100
        
        iteration_result["final_amount"] = final_usdt
        iteration_result["profit"] = profit
        iteration_result["profit_percentage"] = profit_percentage
        
        results["iterations"].append(iteration_result)
        
        # Update investment for next iteration
        current_investment = final_usdt
    
    # Calculate overall results
    initial_investment = investment
    final_investment = current_investment
    total_profit = final_investment - initial_investment
    total_profit_percentage = (total_profit / initial_investment) * 100
    
    # Calculate total impact of formatting and fees for first iteration
    if iterations >= 1:
        first_iteration = results["iterations"][0]
        
        # Calculate ideal final amount without any formatting or fees
        ideal_final_amount = investment
        for step in results["steps"]:
            if "ideal_quantity" in step and "price" in step:
                # This is a neutral operation in terms of value
                pass
        
        # Calculate format impact
        format_impact_usdt = 0
        for step in results["steps"]:
            if step["operation"] == f"Buy {asset} with USDT":
                format_impact_usdt += (step["ideal_quantity"] - step["formatted_quantity"]) * step["price"]
            elif step["operation"] == f"Sell {asset} for {route}":
                format_impact_usdt += step["asset_lost_to_format"] * step["price"] * results["steps"][2]["price"]
            elif step["operation"] == f"Sell {route} for USDT":
                format_impact_usdt += step["route_lost_to_format"] * step["price"]
        
        # Calculate fee impact
        fee_impact_usdt = sum(step["fee"] * (step["price"] if i < 2 else 1) for i, step in enumerate(results["steps"]))
        if results["steps"][0]["fee"] > 0:
            fee_impact_usdt = (
                results["steps"][0]["fee"] * results["steps"][0]["price"] + 
                results["steps"][1]["fee"] * results["steps"][2]["price"] + 
                results["steps"][2]["fee"]
            )
        
        # Calculate theoretical return without formatting or fees
        theoretical_return = investment
        actual_return = results["iterations"][0]["final_amount"]
        total_loss = theoretical_return - actual_return
        
        # If total_loss doesn't match format_impact + fee_impact, adjust
        if abs(total_loss - (format_impact_usdt + fee_impact_usdt)) > 0.0001:
            # This is a simplification - in reality we'd need a more complex calculation
            theoretical_return = investment
            for step in results["steps"]:
                if "price" in step:
                    # This is just to maintain the same value through conversions
                    pass
            
            format_impact_usdt = 0
            for step in results["steps"]:
                if "quantity_lost_to_format" in step or "asset_lost_to_format" in step or "route_lost_to_format" in step:
                    # Calculate impact in USDT terms
                    if step["operation"] == f"Buy {asset} with USDT":
                        format_impact_usdt += step["quantity_lost_to_format"] * step["price"]
                    elif step["operation"] == f"Sell {asset} for {route}":
                        format_impact_usdt += step["asset_lost_to_format"] * step["price"] * results["steps"][2]["price"]
                    elif step["operation"] == f"Sell {route} for USDT":
                        format_impact_usdt += step["route_lost_to_format"] * step["price"]
            
            fee_impact_usdt = investment - actual_return - format_impact_usdt
    else:
        format_impact_usdt = 0
        fee_impact_usdt = 0
    
    # Summary information
    results["summary"] = {
        "initial_investment": initial_investment,
        "final_return": final_investment,
        "total_profit": total_profit,
        "total_profit_percentage": total_profit_percentage,
        "iterations_completed": iterations,
        "average_profit_per_iteration": total_profit / iterations if iterations > 0 else 0,
        "average_profit_percentage_per_iteration": total_profit_percentage / iterations if iterations > 0 else 0,
        "total_fees": sum(step["fee"] for iteration in results["iterations"] for step in iteration["steps"]) if include_fees else 0,
        "total_format_impact_first_iteration": format_impact_usdt,
        "format_impact_percentage_first_iteration": (format_impact_usdt / investment) * 100 if iterations >= 1 and investment > 0 else 0,
        "total_fee_impact_first_iteration": fee_impact_usdt,
        "fee_impact_percentage_first_iteration": (fee_impact_usdt / investment) * 100 if iterations >= 1 and investment > 0 else 0
    }
    
    return results

# Example usage
def print_simulation_results(results):
    """
    Print the simulation results in a readable format
    """
    print(f"===== TRIANGULAR ARBITRAGE SIMULATION =====")
    print(f"Path: USDT → {results['asset']} → {results['route']} → USDT")
    print(f"Initial Investment: {results['investment']} USDT")
    print(f"Iterations: {len(results['iterations'])}")
    
    # Print detailed first iteration
    if results['steps']:
        print("\n----- FIRST ITERATION DETAILS -----")
        for i, step in enumerate(results['steps'], 1):
            print(f"\nStep {i}: {step['operation']}")
            print(f"  Price: {step['price']}")
            
            if i == 1:
                print(f"  Ideal quantity: {step['ideal_quantity']} {results['asset']}")
                print(f"  Formatted quantity: {step['formatted_quantity']} {results['asset']}")
                print(f"  Format impact: {step['quantity_lost_to_format']} {results['asset']} ({step['format_impact_percentage']:.4f}%)")
                print(f"  Cost: {step['cost']} USDT")
                print(f"  Fee: {step['fee']} {results['asset']} ({step['fee_impact_percentage']:.4f}%)")
                print(f"  Received: {step['asset_received']} {results['asset']}")
            elif i == 2:
                print(f"  Ideal sell quantity: {step['ideal_asset_to_sell']} {results['asset']}")
                print(f"  Formatted sell quantity: {step['formatted_asset_to_sell']} {results['asset']}")
                print(f"  Format impact: {step['asset_lost_to_format']} {results['asset']} ({step['format_impact_percentage']:.4f}%)")
                print(f"  Ideal received: {step['ideal_route_quantity']} {results['route']}")
                print(f"  Fee: {step['fee']} {results['route']} ({step['fee_impact_percentage']:.4f}%)")
                print(f"  Received: {step['route_received']} {results['route']}")
            elif i == 3:
                print(f"  Ideal sell quantity: {step['ideal_route_to_sell']} {results['route']}")
                print(f"  Formatted sell quantity: {step['formatted_route_to_sell']} {results['route']}")
                print(f"  Format impact: {step['route_lost_to_format']} {results['route']} ({step['format_impact_percentage']:.4f}%)")
                print(f"  Ideal return: {step['ideal_usdt_return']} USDT")
                print(f"  Fee: {step['fee']} USDT ({step['fee_impact_percentage']:.4f}%)")
                print(f"  Received: {step['usdt_received']} USDT")
    
    # Print iteration summary
    if results['iterations']:
        print("\n----- ITERATIONS SUMMARY -----")
        for iteration in results['iterations']:
            print(f"Iteration {iteration['iteration']}: {iteration['starting_amount']:.6f} USDT → {iteration['final_amount']:.6f} USDT " +
                  f"(Profit: {iteration['profit']:.6f} USDT, {iteration['profit_percentage']:.4f}%)")
    
    print("\n----- OVERALL SUMMARY -----")
    summary = results['summary']
    print(f"Initial investment: {summary['initial_investment']} USDT")
    print(f"Final return after {summary['iterations_completed']} iterations: {summary['final_return']:.6f} USDT")
    print(f"Total profit/loss: {summary['total_profit']:.6f} USDT ({summary['total_profit_percentage']:.4f}%)")
    print(f"Average profit per iteration: {summary['average_profit_per_iteration']:.6f} USDT ({summary['average_profit_percentage_per_iteration']:.4f}%)")
    
    # Show both format and fee impacts
    print("\n----- IMPACT ANALYSIS (FIRST ITERATION) -----")
    print(f"Format impact: {summary['total_format_impact_first_iteration']:.6f} USDT ({summary['format_impact_percentage_first_iteration']:.4f}%)")
    print(f"Fee impact: {summary['total_fee_impact_first_iteration']:.6f} USDT ({summary['fee_impact_percentage_first_iteration']:.4f}%)")
    print(f"Total impact: {summary['total_format_impact_first_iteration'] + summary['total_fee_impact_first_iteration']:.6f} USDT " +
          f"({summary['format_impact_percentage_first_iteration'] + summary['fee_impact_percentage_first_iteration']:.4f}%)")
    print("=====================================")

# Example of how to use the function:
simulation = simulate_triangular_arbitrage(client, 1000, 'STEEM', 'BTC', include_fees=True, iterations=50)
print_simulation_results(simulation)

===== TRIANGULAR ARBITRAGE SIMULATION =====
Path: USDT → STEEM → BTC → USDT
Initial Investment: 1000 USDT
Iterations: 50

----- FIRST ITERATION DETAILS -----

Step 1: Buy STEEM with USDT
  Price: 0.1326
  Ideal quantity: 7541.478129713424 STEEM
  Formatted quantity: 7541.4 STEEM
  Format impact: 0.07812971342445962 STEEM (0.0010%)
  Cost: 999.9896399999999 USDT
  Fee: 7.541399999999999 STEEM (0.1000%)
  Received: 7533.8586 STEEM

Step 2: Sell STEEM for BTC
  Price: 1.59e-06
  Ideal sell quantity: 7533.8586 STEEM
  Formatted sell quantity: 7533.0 STEEM
  Format impact: 0.8585999999995693 STEEM (0.0114%)
  Ideal received: 0.011978835174 BTC
  Fee: 1.1977470000000001e-05 BTC (0.1000%)
  Received: 0.01196549253 BTC

Step 3: Sell BTC for USDT
  Price: 83765.81
  Ideal sell quantity: 0.01196549253 BTC
  Formatted sell quantity: 0.01196 BTC
  Format impact: 5.492529999999787e-06 BTC (0.0459%)
  Ideal return: 1002.2991738243993 USDT
  Fee: 1.0018390876 USDT (0.1000%)
  Received: 1000.837248512